In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import time

res = requests.get("https://www.fantasypros.com/nfl/rankings/rookies.php")
soup = BS(res.content, "html.parser")

In [2]:
# var adpData
scripts = soup.find_all("script")
# script

In [3]:
script = [script for script in scripts if "adpData" in str(script)][0]
# script

In [4]:
script_lines = str(script).split("\n")
# script_lines

In [5]:
ecrData = [line for line in script_lines if "ecrData" in line][0]
# ecrData

In [6]:
player_data = json.loads(ecrData.replace("    var ecrData = ","").replace(";",""))

player_data = player_data['players']

# player_data

In [7]:
data_list = []

for player in player_data:
    player_name = player['player_name']
    player_id = player['player_id']
    position = player['player_positions']
    team = player['player_team_id']
    age = player['player_age']
    ecr = player['rank_ecr']
    
    pfjsonp = "680"
    api_key = "2efb16f8f36d0897c3e188b67e3292d5"
    url = f"https://partners.fantasypros.com/api/v1/nfl-player.php?api_key={api_key}&callback=pfjsonp_{pfjsonp}&player={player_id}&week=current&scoring=STD"
    res = requests.get(url).text
    player_stats = json.loads(res.replace(f"pfjsonp_{pfjsonp}(","").replace(");",""))
    data = player_stats['stats'][0]['stats']
    data['Player'] = player_name
    data['position'] = position
    data['player_team_id'] = team
    data['player_age'] = age
    data['rank_ecr'] = ecr
    
    data_list.append(data)

In [8]:
stats_df = pd.DataFrame(data_list)
stats_df

,POINTS,ATT,YDS,TDS,REC,Player,position,player_team_id,player_age,rank_ecr,CMP,CMP %,INTS
0,176.8,219.6,991.7,6.7,39.3,Breece Hall,RB,NYJ,21,1,NaN,NaN,NaN
1,113.5,NaN,863.7,4.7,65.5,Drake London,WR,ATL,21,2,NaN,NaN,NaN
2,105.6,164.8,643.8,4.6,17.6,Ken Walker III,RB,SEA,21,3,NaN,NaN,NaN
3,116.2,NaN,834.8,5,58.7,Treylon Burks,WR,TEN,22,4,NaN,NaN,NaN
4,70.3,NaN,526,3.1,38.3,Jameson Williams,WR,DET,21,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0,0,0,0,0,Master Teague,RB,FA,22,90,NaN,NaN,NaN
90,0,NaN,0,0,0,Ty Fryfogle,WR,FA,23,91,NaN,NaN,NaN
91,0,0,0,0,NaN,Jack Coan,QB,IND,23,92,0,0,0
92,0,NaN,0,0,0,Reggie Roberson,WR,FA,None,93,NaN,NaN,NaN


In [9]:
stats_df.to_csv('raw_rookie_stats.csv')

In [10]:
index_df = stats_df.set_index('Player').reset_index()
index_df

,Player,POINTS,ATT,YDS,TDS,REC,position,player_team_id,player_age,rank_ecr,CMP,CMP %,INTS
0,Breece Hall,176.8,219.6,991.7,6.7,39.3,RB,NYJ,21,1,NaN,NaN,NaN
1,Drake London,113.5,NaN,863.7,4.7,65.5,WR,ATL,21,2,NaN,NaN,NaN
2,Ken Walker III,105.6,164.8,643.8,4.6,17.6,RB,SEA,21,3,NaN,NaN,NaN
3,Treylon Burks,116.2,NaN,834.8,5,58.7,WR,TEN,22,4,NaN,NaN,NaN
4,Jameson Williams,70.3,NaN,526,3.1,38.3,WR,DET,21,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Master Teague,0,0,0,0,0,RB,FA,22,90,NaN,NaN,NaN
90,Ty Fryfogle,0,NaN,0,0,0,WR,FA,23,91,NaN,NaN,NaN
91,Jack Coan,0,0,0,0,NaN,QB,IND,23,92,0,0,0
92,Reggie Roberson,0,NaN,0,0,0,WR,FA,None,93,NaN,NaN,NaN


In [11]:
drop_df = index_df.drop(columns=['CMP','CMP %','INTS'])
drop_df

,Player,POINTS,ATT,YDS,TDS,REC,position,player_team_id,player_age,rank_ecr
0,Breece Hall,176.8,219.6,991.7,6.7,39.3,RB,NYJ,21,1
1,Drake London,113.5,NaN,863.7,4.7,65.5,WR,ATL,21,2
2,Ken Walker III,105.6,164.8,643.8,4.6,17.6,RB,SEA,21,3
3,Treylon Burks,116.2,NaN,834.8,5,58.7,WR,TEN,22,4
4,Jameson Williams,70.3,NaN,526,3.1,38.3,WR,DET,21,5
...,...,...,...,...,...,...,...,...,...,...
89,Master Teague,0,0,0,0,0,RB,FA,22,90
90,Ty Fryfogle,0,NaN,0,0,0,WR,FA,23,91
91,Jack Coan,0,0,0,0,NaN,QB,IND,23,92
92,Reggie Roberson,0,NaN,0,0,0,WR,FA,None,93


In [12]:
# drop_df.loc[drop_df['position'] == 'WR']

In [13]:
# drop_df.loc[drop_df['position'] == 'RB']

In [14]:
reorder_df = drop_df.fillna(0)
# reorder_df

In [15]:
reorder_df = reorder_df[['Player','player_team_id', 'REC', 'YDS', 'TDS','POINTS','position','rank_ecr']]
reorder_df

,Player,player_team_id,REC,YDS,TDS,POINTS,position,rank_ecr
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,1
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,2
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,3
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,4
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,5
...,...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB,90
90,Ty Fryfogle,FA,0,0,0,0,WR,91
91,Jack Coan,IND,0,0,0,0,QB,92
92,Reggie Roberson,FA,0,0,0,0,WR,93


In [16]:
rename_df = reorder_df.rename(columns={'player_team_id':'2021 Tm','REC':'2021 Rec', 'YDS':'2021 RushingYds','TDS':'2021 ReceivingTD', 'POINTS':'2021 Fantasy Points','position':'Pos','rank_ecr':'AVG'})

rename_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,1
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,2
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,3
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,4
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,5
...,...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB,90
90,Ty Fryfogle,FA,0,0,0,0,WR,91
91,Jack Coan,IND,0,0,0,0,QB,92
92,Reggie Roberson,FA,0,0,0,0,WR,93


In [17]:
cleaned_df = rename_df.loc[(rename_df['2021 Fantasy Points'].astype(float) > 0)]
cleaned_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,1
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,2
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,3
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,4
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,5
...,...,...,...,...,...,...,...,...
72,Jalen Nailor,MIN,3.4,39.9,0.3,5.8,WR,73
73,Samori Toure,GB,6.8,80.6,0.7,12.4,WR,74
74,Michael Woods II,FA,3.9,60.5,0.3,8.1,WR,75
76,Tyler Goodson,GB,9.1,198.1,1.9,40.2,RB,77


In [18]:
QB_df = cleaned_df.loc[(cleaned_df['Pos']== 'QB')].drop(columns='2021 Rec').rename(columns={'2021 ReceivingTD':'Touchdowns21'})
QB_df

,Player,2021 Tm,2021 RushingYds,Touchdowns21,2021 Fantasy Points,Pos,AVG
19,Kenny Pickett,PIT,1559,9.2,102.4,QB,20
23,Malik Willis,TEN,188.7,1,12.1,QB,24
27,Desmond Ridder,ATL,1189.8,6.2,80.5,QB,28
40,Sam Howell,WAS,109.2,0.8,8.1,QB,41


In [19]:
QB_df.to_csv('Rookies_QB.csv')

In [20]:
Other_df = cleaned_df.loc[(cleaned_df['Pos']!= 'QB')]
Other_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,1
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,2
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,3
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,4
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,5
...,...,...,...,...,...,...,...,...
72,Jalen Nailor,MIN,3.4,39.9,0.3,5.8,WR,73
73,Samori Toure,GB,6.8,80.6,0.7,12.4,WR,74
74,Michael Woods II,FA,3.9,60.5,0.3,8.1,WR,75
76,Tyler Goodson,GB,9.1,198.1,1.9,40.2,RB,77


In [21]:
Other_df.to_csv('Rookies_RB_WR_TE.csv')